**Important: Before you start, please add your name and id to the notebook file name and add it here as well**

Example: "notebook1_John_Doe_123123123.ipynb"

**Names:**

1. Ofir Gormes 208575068

2. Eden Bresler 209424589

# Part 1

# Scoring matrix BLOSUM62

Install `biopython` using the following steps:
1. Run "Anaconda Prompt" as administrator (from the Start Menu)
2. Type `conda install -c anaconda biopython`

if you're using Google Colab, install `biopython` using the following command:
`!pip install biopython` (uncomment it in the command below)

Obtain BLOSUM62 scoring matrix for amino acids as follows:

In [1]:
# uncomment the following line if you're using Google Colab:
!pip install biopython

from Bio.Align import substitution_matrices
matrix = substitution_matrices.load("BLOSUM62")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.4 MB/s eta 0:00:00


For example, the score for aligning amino acid `S` versus `P` is:

In [2]:
matrix['S', 'P']

-1.0

In [3]:
matrix['I', 'I']

4.0

In [4]:
matrix['H', 'K']

-1.0

# BLAST
The input for BLAST is a **Database (DB)** sequence, and one or more **query** sequences.  
In addition, a scoring matrix and parameters $k$, $T$, and $X$.

## 1. **Database pre-process**:
  
  Mapping each $k$-mer (a substrings of length $k$) from the **DB** to all its indexes in the **DB** sequence.
  This is done once for the DB sequence, and used for multiple queries.
  
  Complete the function `build_db` that creates a dictionary as follows:   
  **key** - a $k$-mer from `db`, **value** - a list of start indexes of the $k$-mer in `db`

In [13]:
def build_db(db, k):

    db_dict = {}
    for i in range(0, len(db) - k + 1):
      if db[i: i+k] not in db_dict.keys():
        db_dict[db[i: i+k]] = [i]
      else:
        db_dict[db[i: i+k]].append(i)


    return db_dict

In [16]:
db_dict = build_db('AACGTAAAC', 3)

In [17]:
assert db_dict == {'AAC': [0, 6], 'ACG': [1], 'CGT': [2], 'GTA': [3], 'TAA': [4], 'AAA': [5]}

## 2. **Query processing**:
  
###  i. For each $k$-mer in the query, make a list of neighbor words.
  > **Reminder**: A neighbor word $𝑠′$ of a $k$-mer $𝑠$ is a word of length $|s|=k$, with $score(s,s')\geq T$ (using a scoring matrix). $T$ is a chosen threshold.


#### First, a helper function.   
Complete the function `align` that gets two sequences of the same length and a scoring matrix, and returns the score of alignment without indels.

In [18]:
def align(seq1, seq2, scoring_matrix):
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be with the same length")
    score = 0
    for i in range(len(seq1)):
      score += scoring_matrix[seq1[i], seq2[i]]
    return score

In [19]:
assert align('YLKSHFCTA', 'YLNAKKFVH', matrix) == 4

#### Finding neighbors
Let's look at the alphabet of amino acids

In [20]:
alphabet = matrix.alphabet

In [21]:
alphabet

'ARNDCQEGHILKMFPSTWYVBZX*'

We will use the first 20 amino-acid symbols, the last 4 signify a combination of two or more amino acids

In [22]:
ALPHABET = matrix.alphabet[:20]
ALPHABET

'ARNDCQEGHILKMFPSTWYV'

Complete the function `find_neighbors_rec` that finds all neighbors of a $k$-mer

> Description of `find_neighbors_rec`:    
In each step of the recursion, assume that `neighbor[pos:] == kmer[pos:]`   
**Stop condition**: If `pos` is equal to the length of the `kmer`, we add the `neighbor` to `neighbors` list   
**Otherwise**, we will try to replace the char in index `pos` with each letter from the `alphabet` (including itself)    
The new score will be obtained by removing the alignment score of `kmer[pos]` with itself, and adding the alignment score of `kmer[pos]` with the new letter  
If the new score is $>=T$, we will continue with the recursion (and go to the next position)

In [33]:
def find_neighbors(kmer, scoring_matrix, alphabet, T):

    neighbors = []
    max_score = align(kmer, kmer, scoring_matrix)

    if max_score >= T:
        find_neighbors_rec(kmer, kmer, 0, max_score, alphabet, neighbors, scoring_matrix, T)

    return neighbors

def find_neighbors_rec(kmer, neighbor, pos, curr_score, alphabet, neighbors, scoring_matrix, T):
    if pos == len(kmer):
      neighbors.append(neighbor)
    else:
      for letter in alphabet:
        neighbor = neighbor[:pos] + letter + neighbor[pos+1:]
        new_score = curr_score - align(kmer[pos], kmer[pos], scoring_matrix) + align(kmer[pos], letter, scoring_matrix)
        if new_score >= T:
          find_neighbors_rec(kmer, neighbor, pos + 1, new_score, alphabet, neighbors, scoring_matrix, T)


These are all possible neighbors of `YI`

In [34]:
neighbors = find_neighbors('YI', matrix, ALPHABET, -50)

This number should be equal to $20^2=400$

In [35]:
assert len(neighbors) == 400

In [36]:
neighbors[:10]

['AA', 'AR', 'AN', 'AD', 'AC', 'AQ', 'AE', 'AG', 'AH', 'AI']

In [37]:
neighbors = find_neighbors('YI', matrix, ALPHABET, 6)

In [38]:
neighbors

['HI', 'FI', 'FV', 'WI', 'YA', 'YC', 'YI', 'YL', 'YM', 'YF', 'YT', 'YY', 'YV']

In [39]:
assert set(neighbors) == set(['HI', 'FI', 'FV', 'WI', 'YA', 'YC', 'YI', 'YL', 'YM', 'YF', 'YT', 'YY', 'YV'])

Let's compare the running time of going over all possible neighbors of a $k$-mer of length 5 (and then extracting only neighbors with score $\geq T$)

We will get all possible neighbors of `YIIMV` by setting $T=-100$

In [40]:
from datetime import datetime

start_time = datetime.now()

neighbors = find_neighbors('YIIMV', matrix, ALPHABET, -100)

timedelta = datetime.now() - start_time

In [41]:
print(timedelta)

0:00:25.286811


Now we will use our efficient recursion, with $T=20$

In [42]:
start_time = datetime.now()

neighbors = find_neighbors('YIIMV', matrix, ALPHABET, 20)

timedelta = datetime.now() - start_time

In [43]:
print(timedelta)

0:00:00.008597


This is a huge time improvement!   
Remember that we need to find neighbors of each k-mer from the query